In [1]:
import sys
sys.path.insert(0, './src')  # 或者绝对路径：/full/path/to/your_project
from src.xmcdata import *
from model import seq2seqParams, Seq2SeqModel
from dataset import load_xmc_seq2seq_dataset, load_xmc_seq2seq_dataset_from_disk

In [6]:
dataset_name = 'eurlex-4k'
model_name = 't5'
params_path = f'./params/{dataset_name}/{model_name}.json'

In [3]:
data_dir = f"xmc-base/{dataset_name}"

In [4]:
label_map = load_label_text_map(data_dir + "/output-items.txt")
prefix = 'Summarization of this paragraph by unstemmed keyphrases: '

X_trn_text = load_texts(data_dir+"/X.trn.txt")
#X_trn_text = [prefix + text for text in X_trn_text]
Y_trn_feat = load_sparse_matrix(data_dir+"/Y.trn.npz")

Y_trn_text,Y_trn_num = csr_id_to_text(load_sparse_matrix(data_dir+"/Y.trn.npz"),label_map)
Y_trn_text = [",".join(y) for y in Y_trn_text]
Y_trn_text = [prefix + text for text in Y_trn_text]

X_tst_text = load_texts(data_dir+"/X.tst.txt")
Y_tst_feat = load_sparse_matrix(data_dir+"/Y.tst.npz")
#X_tst_text = [prefix + text for text in X_tst_text]
Y_tst_text, Y_tst_num = csr_id_to_text(load_sparse_matrix(data_dir+"/Y.tst.npz"),label_map)
Y_tst_text = [",".join(y) for y in Y_tst_text]
Y_tst_text = [prefix + text for text in Y_tst_text]

In [7]:

args = seq2seqParams.load_config(params_path)

model =  Seq2SeqModel(args)

train_dataset, val_dataset = load_xmc_seq2seq_dataset(
    X_trn_text, Y_trn_text, X_tst_text, Y_tst_text, model.tokenizer,max_length=args.max_input_length,save_dir = data_dir
)
train_dataset, val_dataset = load_xmc_seq2seq_dataset_from_disk(data_dir)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/root/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torch/cuda/__init__.py:287: UserWarning: 
NVIDIA GeForce RTX 5090 with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_50 sm_60 sm_70 sm_75 sm_80 sm_86 sm_90.
If you want to use the NVIDIA GeForce RTX 5090 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(
/root/.pyenv/versions/3.10.13/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

...

/root/.pyenv/versions/3.10.13/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/3865 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/15449 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3865 [00:00<?, ? examples/s]

Datasets saved to xmc-base/eurlex-4k
Loading datasets from xmc-base/eurlex-4k


In [8]:
output_dir = f"output/{dataset_name}/{model_name}"
trainer, _ = model.gen_train(train_dataset, val_dataset,output_dir)

/root/.pyenv/versions/3.10.13/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Training model google-t5/t5-base with output directory output/eurlex-4k/t5
Training arguments: seq2seqParams(model_name_or_path='google-t5/t5-base', max_input_length=128, max_output_length=64, num_train_epochs=5, batch_size=5, learning_rate=5e-05, use_fp16=True, logging_steps=20, weight_decay=0.01, warmup_ratio=0.1, use_tensorboard=True, num_beams=4)


RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [19]:
!pip install sentencepiece

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.5 MB/s eta 0:00:00


In [1]:
from transformers import AutoModelForSeq2SeqLM, T5Tokenizer
import torch
model = AutoModelForSeq2SeqLM.from_pretrained("t5-base", cache_dir="./cache")
tokenizer = T5Tokenizer.from_pretrained("t5-base")

X = "Summarize: how transformers work."
Y = "transformer, attention, encoder"

inputs = tokenizer(X, return_tensors="pt", padding="max_length", max_length=128)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(Y, return_tensors="pt", padding="max_length", max_length=64).input_ids

labels[labels == tokenizer.pad_token_id] = -100

print("✅ vocab_size:", tokenizer.vocab_size)
print("✅ labels max:", torch.max(labels[labels != -100]))

model.to("cuda")
outputs = model(**inputs.to("cuda"), labels=labels.to("cuda"))
print(outputs.loss)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/root/.pyenv/versions/3.10.13/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


✅ vocab_size: 32000
✅ labels max: tensor(23734)
tensor(7.0549, device='cuda:0', grad_fn=<NllLossBackward0>)


In [8]:
output_dir = f"output/{dataset_name}/{model_name}"
model.model.from_pretrained(output_dir)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): Laye

In [ ]:
model.predict(X_tst_text,batch_size=8,max_length=args.max_output_length,num_beams=args.num_beams,output_path=data_dir)

TypeError: unsupported operand type(s) for /: 'str' and 'str'